# Tutorial 2: Game Theory Foundations

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/blockchain-tokens-and-incentives/blob/main/notebooks/tutorial_02_game_theory_foundations.ipynb)

---

> *"A guild member does not cooperate because he is virtuous. He cooperates because, in a repeated game with memory, cooperation is the strategy that maximizes cumulative payoff."*
>
> — Brenn Auster, *On the Alignment of Interest and Obligation* (Year 875)

---

## From Observation to Theory

Tutorial 01 showed the guild's enforcement failure: defection is rational in one-shot games, inspectors can be bribed, and the enforcement paradox means more rules can make things worse.

Auster needed a formal framework. Game theory — the mathematics of strategic interaction — provides exactly that. This tutorial builds the theoretical machinery that makes mechanism design (Tutorial 03) possible.

We will analyze the `game_theory_outcomes.csv` dataset: 30 matches of the iterated Prisoner's Dilemma, each running 10 rounds, with 8 different strategies competing.

## Learning Objectives

In this tutorial, you will:

1. **Formalize the Prisoner's Dilemma** — payoff matrices, strategy spaces, solution concepts
2. **Find Nash equilibria** — strategy profiles where no player can improve by switching
3. **Implement and analyze 8 strategies** — from always-defect to tit-for-tat to pavlov
4. **Run a strategy tournament** — which strategy wins in repeated games?
5. **Understand how repetition transforms the game** — the Folk Theorem and the shadow of the future

**Technical Concepts:**
- Nash equilibrium and dominant strategies
- Iterated Prisoner's Dilemma and Axelrod's tournament
- Tit-for-tat, grim trigger, pavlov
- The Folk Theorem: why cooperation can be sustained

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

game_theory = pd.read_csv(BASE_URL + "game_theory_outcomes.csv")

print(f"Game theory outcomes: {len(game_theory)} rounds")
print(f"Matches: {game_theory['match_id'].nunique()}")
print(f"Rounds per match: {game_theory.groupby('match_id')['round'].max().unique()}")
print(f"Strategies: {sorted(set(game_theory['strategy_a']) | set(game_theory['strategy_b']))}")

## Formalizing the Game

A **game** in game theory has three components:
1. **Players** — the decision makers (two guild traders)
2. **Strategies** — the available actions (cooperate or defect)
3. **Payoffs** — the outcomes for each combination of actions

The Prisoner's Dilemma uses the payoff ordering **T > R > P > S** and **2R > T + S**.

In [ ]:
# Formal payoff matrix
T, R, P, S = 5, 3, 1, 0  # Temptation, Reward, Punishment, Sucker

PAYOFF_MATRIX = {
    ('cooperate', 'cooperate'): (R, R),
    ('cooperate', 'defect'):    (S, T),
    ('defect', 'cooperate'):    (T, S),
    ('defect', 'defect'):       (P, P),
}

print("Prisoner's Dilemma Payoff Matrix:")
print(f"\n  T={T} (Temptation to defect)")
print(f"  R={R} (Reward for mutual cooperation)")
print(f"  P={P} (Punishment for mutual defection)")
print(f"  S={S} (Sucker's payoff)")
print(f"\n  Conditions: T > R > P > S: {T} > {R} > {P} > {S} ✓")
print(f"  Conditions: 2R > T + S: {2*R} > {T+S} ✓ (cooperation is efficient)")

# Display as a proper matrix
print(f"\n{'':20s} {'B: Cooperate':>15s} {'B: Defect':>15s}")
print(f"{'A: Cooperate':20s} {'(R,R)=(3,3)':>15s} {'(S,T)=(0,5)':>15s}")
print(f"{'A: Defect':20s} {'(T,S)=(5,0)':>15s} {'(P,P)=(1,1)':>15s}")

## Nash Equilibrium

A **Nash equilibrium** is a strategy profile where no player can improve their payoff by unilaterally changing their strategy. In the one-shot Prisoner's Dilemma, (Defect, Defect) is the unique Nash equilibrium.

In [ ]:
def find_nash_equilibria(payoff_matrix):
    """Find all pure-strategy Nash equilibria in a 2-player game."""
    actions = ['cooperate', 'defect']
    equilibria = []
    
    for a1, a2 in product(actions, repeat=2):
        pay_a, pay_b = payoff_matrix[(a1, a2)]
        
        # Check: can A improve by switching?
        a_alt = 'defect' if a1 == 'cooperate' else 'cooperate'
        alt_pay_a, _ = payoff_matrix[(a_alt, a2)]
        a_can_improve = alt_pay_a > pay_a
        
        # Check: can B improve by switching?
        b_alt = 'defect' if a2 == 'cooperate' else 'cooperate'
        _, alt_pay_b = payoff_matrix[(a1, b_alt)]
        b_can_improve = alt_pay_b > pay_b
        
        is_nash = not a_can_improve and not b_can_improve
        
        print(f"  ({a1:9s}, {a2:9s}) → payoffs ({pay_a}, {pay_b})")
        print(f"    A switch to {a_alt}: payoff {alt_pay_a} {'> ' + str(pay_a) + ' (improves)' if a_can_improve else '<= ' + str(pay_a) + ' (no improvement)'}")
        print(f"    B switch to {b_alt}: payoff {alt_pay_b} {'> ' + str(pay_b) + ' (improves)' if b_can_improve else '<= ' + str(pay_b) + ' (no improvement)'}")
        print(f"    Nash equilibrium: {'YES ✓' if is_nash else 'no'}")
        print()
        
        if is_nash:
            equilibria.append((a1, a2, pay_a, pay_b))
    
    return equilibria

print("Finding Nash equilibria in the Prisoner's Dilemma:\n")
equilibria = find_nash_equilibria(PAYOFF_MATRIX)
print(f"Nash equilibria: {[(e[0], e[1]) for e in equilibria]}")
print(f"\nThe only Nash equilibrium is (Defect, Defect) with payoffs (1, 1).")
print(f"But (Cooperate, Cooperate) gives (3, 3). The dilemma: the equilibrium is inefficient.")

## Dominant Strategies

A **dominant strategy** is one that gives a better payoff than any alternative, regardless of what the opponent does. In the PD, defection is strictly dominant.

In [ ]:
def check_dominance(payoff_matrix):
    """Check for strictly dominant strategies."""
    actions = ['cooperate', 'defect']
    
    for player_label, idx in [('Player A', 0), ('Player B', 1)]:
        print(f"{player_label}:")
        for my_action in actions:
            payoffs = []
            for opp_action in actions:
                if idx == 0:
                    pay = payoff_matrix[(my_action, opp_action)][idx]
                else:
                    pay = payoff_matrix[(opp_action, my_action)][idx]
                payoffs.append((opp_action, pay))
            print(f"  {my_action:10s} → {payoffs}")
        
        # Check if defect dominates cooperate
        defect_payoffs = [payoff_matrix[('defect', a)][0] if idx == 0 
                          else payoff_matrix[(a, 'defect')][1] for a in actions]
        coop_payoffs = [payoff_matrix[('cooperate', a)][0] if idx == 0 
                        else payoff_matrix[(a, 'cooperate')][1] for a in actions]
        
        dominates = all(d > c for d, c in zip(defect_payoffs, coop_payoffs))
        print(f"  Defect dominates cooperate: {dominates}\n")

check_dominance(PAYOFF_MATRIX)

## The Eight Strategies

The guild members use 8 different strategies. Let's implement each one and understand its logic.

In [ ]:
def always_cooperate(my_history, opp_history, round_num):
    return 'cooperate'

def always_defect(my_history, opp_history, round_num):
    return 'defect'

def tit_for_tat(my_history, opp_history, round_num):
    """Cooperate first, then copy opponent's last move."""
    if round_num == 1:
        return 'cooperate'
    return opp_history[-1]

def generous_tit_for_tat(my_history, opp_history, round_num):
    """Like tit-for-tat, but forgive 10% of defections."""
    if round_num == 1:
        return 'cooperate'
    if opp_history[-1] == 'defect':
        return 'cooperate' if np.random.random() < 0.1 else 'defect'
    return 'cooperate'

def random_strategy(my_history, opp_history, round_num):
    return np.random.choice(['cooperate', 'defect'])

def grim_trigger(my_history, opp_history, round_num):
    """Cooperate until opponent defects once, then defect forever."""
    if 'defect' in opp_history:
        return 'defect'
    return 'cooperate'

def pavlov(my_history, opp_history, round_num):
    """Repeat last action if it paid well; switch otherwise."""
    if round_num == 1:
        return 'cooperate'
    if my_history[-1] == opp_history[-1]:
        return my_history[-1]
    return 'defect' if my_history[-1] == 'cooperate' else 'cooperate'

def suspicious_tit_for_tat(my_history, opp_history, round_num):
    """Like tit-for-tat but starts with defection."""
    if round_num == 1:
        return 'defect'
    return opp_history[-1]

STRATEGIES = {
    'always_cooperate': always_cooperate,
    'always_defect': always_defect,
    'tit_for_tat': tit_for_tat,
    'generous_tit_for_tat': generous_tit_for_tat,
    'random': random_strategy,
    'grim_trigger': grim_trigger,
    'pavlov': pavlov,
    'suspicious_tit_for_tat': suspicious_tit_for_tat,
}

print(f"Implemented {len(STRATEGIES)} strategies:")
for name, fn in STRATEGIES.items():
    print(f"  {name:30s} — {fn.__doc__ or 'No description'}")

## Running a Match

Let's build a match simulator and watch two strategies play against each other.

In [ ]:
def play_match(strategy_a_fn, strategy_b_fn, rounds=10):
    """Play a match between two strategies. Returns round-by-round results."""
    history_a, history_b = [], []
    results = []
    
    for r in range(1, rounds + 1):
        action_a = strategy_a_fn(history_a, history_b, r)
        action_b = strategy_b_fn(history_b, history_a, r)
        payoff_a, payoff_b = PAYOFF_MATRIX[(action_a, action_b)]
        
        history_a.append(action_a)
        history_b.append(action_b)
        
        results.append({
            'round': r, 'action_a': action_a, 'action_b': action_b,
            'payoff_a': payoff_a, 'payoff_b': payoff_b,
            'cumulative_a': sum(rr['payoff_a'] for rr in results) + payoff_a,
            'cumulative_b': sum(rr['payoff_b'] for rr in results) + payoff_b,
        })
    
    return pd.DataFrame(results)

# Classic matchup: tit_for_tat vs always_defect
match = play_match(STRATEGIES['tit_for_tat'], STRATEGIES['always_defect'])

print("Match: tit_for_tat (A) vs always_defect (B)\n")
print(f"{'Round':>5s} {'A Action':>12s} {'B Action':>12s} {'A Pay':>6s} {'B Pay':>6s} {'A Cum':>6s} {'B Cum':>6s}")
for _, row in match.iterrows():
    print(f"{int(row['round']):5d} {row['action_a']:>12s} {row['action_b']:>12s} "
          f"{int(row['payoff_a']):6d} {int(row['payoff_b']):6d} "
          f"{int(row['cumulative_a']):6d} {int(row['cumulative_b']):6d}")

In [ ]:
# Visualize three key matchups
matchups = [
    ('tit_for_tat', 'tit_for_tat', 'TFT vs TFT (mutual cooperation)'),
    ('tit_for_tat', 'always_defect', 'TFT vs Always Defect'),
    ('always_defect', 'always_defect', 'Defect vs Defect (Nash equilibrium)'),
]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for ax, (sa, sb, title) in zip(axes, matchups):
    np.random.seed(42)
    m = play_match(STRATEGIES[sa], STRATEGIES[sb])
    ax.plot(m['round'], m['cumulative_a'], 'o-', label=sa, color='steelblue')
    ax.plot(m['round'], m['cumulative_b'], 's--', label=sb, color='firebrick')
    ax.set_xlabel('Round')
    ax.set_ylabel('Cumulative Payoff')
    ax.set_title(title, fontsize=10)
    ax.legend(fontsize=8)
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Analyzing the Dataset

Our dataset contains 30 matches from the guild's game theory experiments. Let's analyze strategy performance across all matches.

In [ ]:
# Strategy performance from the dataset (final-round cumulative payoffs)
final_rounds = game_theory[game_theory['round'] == 10].copy()

# Collect all payoffs for each strategy (whether played as A or B)
strategy_payoffs = {}
for _, row in final_rounds.iterrows():
    sa, sb = row['strategy_a'], row['strategy_b']
    pa, pb = row['cumulative_payoff_a'], row['cumulative_payoff_b']
    strategy_payoffs.setdefault(sa, []).append(pa)
    strategy_payoffs.setdefault(sb, []).append(pb)

# Compute stats
perf = pd.DataFrame([
    {'strategy': s, 'avg_payoff': np.mean(pays), 'std': np.std(pays),
     'min': np.min(pays), 'max': np.max(pays), 'matches': len(pays)}
    for s, pays in strategy_payoffs.items()
]).sort_values('avg_payoff', ascending=False)

print("Strategy Tournament Results (from dataset):")
print(f"{'Strategy':30s} {'Avg':>6s} {'Std':>6s} {'Min':>5s} {'Max':>5s} {'N':>4s}")
for _, row in perf.iterrows():
    print(f"{row['strategy']:30s} {row['avg_payoff']:6.1f} {row['std']:6.1f} "
          f"{int(row['min']):5d} {int(row['max']):5d} {int(row['matches']):4d}")

In [ ]:
# Visualize strategy rankings
plt.figure(figsize=(10, 6))
colors = ['#2ecc71' if 'cooperate' in s or 'tit' in s or 'pavlov' in s or 'grim' in s 
          else '#e74c3c' if 'defect' in s else '#f39c12'
          for s in perf['strategy']]
plt.barh(perf['strategy'], perf['avg_payoff'], xerr=perf['std'],
         color=colors, alpha=0.8, edgecolor='black', capsize=3)
plt.xlabel('Average Cumulative Payoff (10 rounds)')
plt.title('Strategy Tournament: Who Wins the Guild Game?')
plt.axvline(x=10, color='gray', linestyle=':', alpha=0.5, label='Mutual defection baseline (10)')
plt.axvline(x=30, color='gray', linestyle='--', alpha=0.5, label='Mutual cooperation optimum (30)')
plt.legend()
plt.tight_layout()
plt.show()

print("Green = cooperative strategies | Red = defective | Orange = mixed")

## Round-by-Round Dynamics

The aggregate scores tell part of the story. The round-by-round dynamics reveal how strategies interact over time.

In [ ]:
# Cooperation rate over rounds
round_coop = game_theory.groupby('round').agg(
    coop_a=('action_a', lambda x: (x == 'cooperate').mean()),
    coop_b=('action_b', lambda x: (x == 'cooperate').mean()),
).reset_index()
round_coop['avg_cooperation'] = (round_coop['coop_a'] + round_coop['coop_b']) / 2

plt.figure(figsize=(10, 5))
plt.plot(round_coop['round'], round_coop['avg_cooperation'], 'o-',
         color='steelblue', linewidth=2, markersize=8)
plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
plt.xlabel('Round')
plt.ylabel('Cooperation Rate')
plt.title('Average Cooperation Rate Over Rounds (All 30 Matches)')
plt.ylim(0, 1)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Round 1 cooperation: {round_coop.iloc[0]['avg_cooperation']:.2%}")
print(f"Round 10 cooperation: {round_coop.iloc[-1]['avg_cooperation']:.2%}")

In [ ]:
# Per-strategy cooperation evolution
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
strategies_list = sorted(game_theory['strategy_a'].unique())

for ax, strat in zip(axes.flat, strategies_list):
    mask_a = game_theory['strategy_a'] == strat
    mask_b = game_theory['strategy_b'] == strat
    
    coop_as_a = game_theory[mask_a].groupby('round')['action_a'].apply(
        lambda x: (x == 'cooperate').mean())
    coop_as_b = game_theory[mask_b].groupby('round')['action_b'].apply(
        lambda x: (x == 'cooperate').mean())
    
    all_coop = pd.concat([coop_as_a, coop_as_b]).groupby(level=0).mean()
    
    ax.plot(all_coop.index, all_coop.values, 'o-', color='steelblue', markersize=4)
    ax.set_title(strat.replace('_', ' '), fontsize=9)
    ax.set_ylim(-0.05, 1.05)
    ax.set_xlabel('Round', fontsize=8)
    ax.grid(alpha=0.3)

plt.suptitle('Cooperation Rate by Strategy Over Rounds', fontsize=12)
plt.tight_layout()
plt.show()

## Full Round-Robin Tournament

Let's run a complete round-robin where every strategy plays every other strategy (and itself) over 10 rounds. This follows Axelrod's famous tournament design.

In [ ]:
# Full round-robin tournament
np.random.seed(875)  # Year of Auster's alignment paper

tournament_results = {}
matchup_details = []

for sa_name, sa_fn in STRATEGIES.items():
    total_payoff = 0
    n_matches = 0
    for sb_name, sb_fn in STRATEGIES.items():
        match_df = play_match(sa_fn, sb_fn, rounds=10)
        final_payoff_a = match_df.iloc[-1]['cumulative_a']
        final_payoff_b = match_df.iloc[-1]['cumulative_b']
        total_payoff += final_payoff_a
        n_matches += 1
        matchup_details.append({
            'strategy_a': sa_name, 'strategy_b': sb_name,
            'payoff_a': final_payoff_a, 'payoff_b': final_payoff_b
        })
    tournament_results[sa_name] = total_payoff / n_matches

# Sort by performance
sorted_results = sorted(tournament_results.items(), key=lambda x: x[1], reverse=True)

print("Round-Robin Tournament Results:")
print(f"{'Rank':>4s} {'Strategy':30s} {'Avg Payoff':>10s}")
for rank, (strategy, avg) in enumerate(sorted_results, 1):
    marker = ' ← WINNER' if rank == 1 else ''
    print(f"{rank:4d} {strategy:30s} {avg:10.1f}{marker}")

In [ ]:
# Matchup heatmap
matchup_df = pd.DataFrame(matchup_details)
pivot = matchup_df.pivot(index='strategy_a', columns='strategy_b', values='payoff_a')

# Sort by tournament ranking
rank_order = [s for s, _ in sorted_results]
pivot = pivot.reindex(index=rank_order, columns=rank_order)

fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(pivot.values, cmap='RdYlGn', aspect='auto')
ax.set_xticks(range(len(rank_order)))
ax.set_yticks(range(len(rank_order)))
ax.set_xticklabels([s.replace('_', '\n') for s in rank_order], fontsize=7, rotation=45, ha='right')
ax.set_yticklabels([s.replace('_', ' ') for s in rank_order], fontsize=8)
ax.set_xlabel('Opponent')
ax.set_ylabel('Player')
ax.set_title('Payoff Heatmap: Row Player vs Column Opponent')

for i in range(len(rank_order)):
    for j in range(len(rank_order)):
        ax.text(j, i, f"{pivot.values[i,j]:.0f}", ha='center', va='center', fontsize=7)

plt.colorbar(im, ax=ax, label='Cumulative Payoff (10 rounds)')
plt.tight_layout()
plt.show()

## The Folk Theorem: Why Cooperation Survives

In a one-shot game, defection dominates. But the **Folk Theorem** proves that in infinitely repeated games (or games with unknown end), *any* outcome where both players earn at least their minimax payoff can be sustained as a Nash equilibrium — including mutual cooperation.

The key parameter is the **discount factor** (δ): how much players value future payoffs relative to present ones.

In [ ]:
# The Folk Theorem condition for cooperation via tit-for-tat:
# Cooperation is sustained if discount factor δ ≥ (T - R) / (T - P)

delta_threshold = (T - R) / (T - P)

print(f"Folk Theorem threshold for Prisoner's Dilemma cooperation:")
print(f"  δ ≥ (T - R) / (T - P) = ({T} - {R}) / ({T} - {P}) = {delta_threshold:.2f}")
print(f"\nIf players value future payoffs at ≥ {delta_threshold:.0%} of present payoffs,")
print(f"tit-for-tat sustains cooperation as a Nash equilibrium.")

# Visualize: cooperation viability by discount factor
deltas = np.linspace(0, 1, 100)
coop_payoff = np.where(deltas >= delta_threshold, R / (1 - deltas + 1e-10), P / (1 - deltas + 1e-10))
defect_payoff = np.full_like(deltas, P / (1 - deltas[50] + 1e-10))  # Always defect baseline

# Simpler: expected per-round payoff under cooperation vs defection
coop_per_round = R  # 3 per round if both cooperate
defect_per_round = P  # 1 per round if both defect
tempt_first_round = T  # 5 for defecting against cooperator

# Value of defecting against tit-for-tat: get T first round, then P forever
defect_values = [tempt_first_round + defect_per_round * d / (1 - d + 1e-10) 
                 for d in deltas]
# Value of cooperating with tit-for-tat: get R every round
coop_values = [coop_per_round / (1 - d + 1e-10) for d in deltas]

plt.figure(figsize=(10, 5))
plt.plot(deltas, coop_values, '-', color='steelblue', linewidth=2, label='Cooperate (R every round)')
plt.plot(deltas, defect_values, '--', color='firebrick', linewidth=2, label='Defect once then punished')
plt.axvline(x=delta_threshold, color='black', linestyle=':', label=f'δ* = {delta_threshold:.2f}')
plt.xlabel('Discount Factor (δ)')
plt.ylabel('Total Discounted Payoff')
plt.title('The Folk Theorem: When Cooperation Becomes Self-Enforcing')
plt.legend()
plt.ylim(0, 100)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## The Dens Interference

Our dataset includes a `dens_interference` flag — a 5% chance that a player's intended action gets flipped. This models the instability near the Dens and has real implications: in a world with noise, forgiving strategies outperform strict ones.

In [ ]:
# Dens interference analysis
interference = game_theory[game_theory['dens_interference'] == True]
no_interference = game_theory[game_theory['dens_interference'] == False]

print(f"Rounds with Dens interference: {len(interference)} ({len(interference)/len(game_theory):.1%})")
print(f"Rounds without: {len(no_interference)}")

# Does interference affect cooperation rates?
intf_coop = (interference['action_a'] == 'cooperate').mean()
no_intf_coop = (no_interference['action_a'] == 'cooperate').mean()
print(f"\nCooperation rate with interference: {intf_coop:.2%}")
print(f"Cooperation rate without: {no_intf_coop:.2%}")

# This matters for strategy selection:
# Grim trigger is CATASTROPHIC with noise (one accidental defect = permanent punishment)
# Generous tit-for-tat is robust (forgives occasional defections)
print(f"\nIn noisy environments (near the Dens), forgiving strategies survive.")
print(f"Grim trigger collapses under noise. Generous tit-for-tat thrives.")

In [ ]:
# Simulate: grim_trigger vs generous_tit_for_tat with noise
def play_match_with_noise(sa_fn, sb_fn, rounds=10, noise=0.05):
    """Play a match with random action flips (Dens interference)."""
    history_a, history_b = [], []
    results = []
    for r in range(1, rounds + 1):
        action_a = sa_fn(history_a, history_b, r)
        action_b = sb_fn(history_b, history_a, r)
        if np.random.random() < noise:
            action_a = 'defect' if action_a == 'cooperate' else 'cooperate'
        if np.random.random() < noise:
            action_b = 'defect' if action_b == 'cooperate' else 'cooperate'
        pa, pb = PAYOFF_MATRIX[(action_a, action_b)]
        history_a.append(action_a)
        history_b.append(action_b)
        cum_a = sum(rr['payoff_a'] for rr in results) + pa
        cum_b = sum(rr['payoff_b'] for rr in results) + pb
        results.append({'round': r, 'action_a': action_a, 'action_b': action_b,
                        'payoff_a': pa, 'payoff_b': pb,
                        'cumulative_a': cum_a, 'cumulative_b': cum_b})
    return pd.DataFrame(results)

# 100 runs with 5% noise
grim_scores, generous_scores = [], []
for _ in range(100):
    m = play_match_with_noise(STRATEGIES['grim_trigger'], STRATEGIES['grim_trigger'], noise=0.05)
    grim_scores.append(m.iloc[-1]['cumulative_a'])
    m = play_match_with_noise(STRATEGIES['generous_tit_for_tat'], STRATEGIES['generous_tit_for_tat'], noise=0.05)
    generous_scores.append(m.iloc[-1]['cumulative_a'])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.hist(grim_scores, bins=15, color='firebrick', alpha=0.7, edgecolor='black')
ax1.axvline(np.mean(grim_scores), color='black', linestyle='--')
ax1.set_title(f'Grim Trigger vs Grim Trigger (5% noise)\navg={np.mean(grim_scores):.1f}')
ax1.set_xlabel('Cumulative Payoff')

ax2.hist(generous_scores, bins=15, color='steelblue', alpha=0.7, edgecolor='black')
ax2.axvline(np.mean(generous_scores), color='black', linestyle='--')
ax2.set_title(f'Generous TFT vs Generous TFT (5% noise)\navg={np.mean(generous_scores):.1f}')
ax2.set_xlabel('Cumulative Payoff')

plt.tight_layout()
plt.show()
print(f"Noise devastates grim_trigger but barely affects generous_tit_for_tat.")

## Auster's Takeaway: The Requirements for Cooperation

Game theory tells Auster exactly what conditions sustain cooperation:

1. **Repeated interaction** — one-shot games incentivize defection; repeated games allow reputation
2. **Memory** — players must remember past interactions (tit-for-tat needs history)
3. **Sufficiently long shadow of the future** — discount factor δ ≥ (T-R)/(T-P)
4. **Noise tolerance** — forgiving strategies survive in imperfect environments

The blockchain implication: **staking creates all four conditions.** Locked tokens ensure long-term engagement (repetition). On-chain history provides memory. Long lockup periods create a shadow of the future. Slashing with appeal mechanisms provides noise tolerance.

## Exercises

### Exercise 1: The Population Ecology

In Axelrod's tournament, strategies that do well attract followers. Simulate 5 generations where the worst-performing strategy is eliminated each generation. Which strategy is the last one standing?

In [ ]:
# Population ecology simulation
np.random.seed(867)
alive = list(STRATEGIES.keys())

print("Population ecology (eliminate worst each generation):\n")
for gen in range(1, 6):
    if len(alive) <= 1:
        break
    scores = {}
    for sa in alive:
        total = 0
        for sb in alive:
            m = play_match(STRATEGIES[sa], STRATEGIES[sb])
            total += m.iloc[-1]['cumulative_a']
        scores[sa] = total / len(alive)
    
    sorted_scores = sorted(scores.items(), key=lambda x: x[1])
    eliminated = sorted_scores[0][0]
    alive.remove(eliminated)
    
    print(f"Generation {gen}: eliminated {eliminated} (avg={sorted_scores[0][1]:.1f})")
    for s, sc in sorted(scores.items(), key=lambda x: -x[1]):
        marker = ' ← ELIMINATED' if s == eliminated else ''
        print(f"    {s:30s} {sc:6.1f}{marker}")
    print()

print(f"Last strategies standing: {alive}")

### Exercise 2: Custom Strategy

Design a strategy that beats tit-for-tat in a noisy environment. Test it against all 8 strategies with 5% noise.

In [ ]:
def custom_strategy(my_history, opp_history, round_num):
    """Your custom strategy: cooperate first, forgive one defection, retaliate on two."""
    if round_num <= 2:
        return 'cooperate'
    # Look at last 2 opponent moves
    if opp_history[-1] == 'defect' and opp_history[-2] == 'defect':
        return 'defect'  # Two defections in a row → retaliate
    return 'cooperate'  # Otherwise forgive

# Test against all strategies with noise
np.random.seed(42)
custom_scores = {}
for name, fn in STRATEGIES.items():
    payoffs = []
    for _ in range(50):  # 50 runs for statistical significance
        m = play_match_with_noise(custom_strategy, fn, noise=0.05)
        payoffs.append(m.iloc[-1]['cumulative_a'])
    custom_scores[name] = np.mean(payoffs)

print("Custom strategy vs all opponents (5% noise, 50 runs each):")
for opp, score in sorted(custom_scores.items(), key=lambda x: -x[1]):
    print(f"  vs {opp:30s} avg payoff: {score:.1f}")
print(f"\nOverall average: {np.mean(list(custom_scores.values())):.1f}")

### Exercise 3: The Discount Factor Experiment

Plot how the tournament rankings change as the discount factor (game length) varies from 2 rounds to 50 rounds. At what game length does tit-for-tat overtake always-defect?

In [ ]:
# Tournament rankings by game length
np.random.seed(42)
lengths = [2, 5, 10, 20, 50]
length_rankings = {s: [] for s in STRATEGIES}

for n_rounds in lengths:
    scores = {}
    for sa_name, sa_fn in STRATEGIES.items():
        total = 0
        for sb_name, sb_fn in STRATEGIES.items():
            m = play_match(sa_fn, sb_fn, rounds=n_rounds)
            total += m.iloc[-1]['cumulative_a']
        scores[sa_name] = total / len(STRATEGIES)
    for s, sc in scores.items():
        length_rankings[s].append(sc)

plt.figure(figsize=(12, 6))
for strategy, scores in length_rankings.items():
    style = '-' if 'tit' in strategy or 'pavlov' in strategy else '--'
    plt.plot(lengths, scores, f'o{style}', label=strategy, linewidth=1.5)
plt.xlabel('Game Length (rounds)')
plt.ylabel('Average Tournament Score')
plt.title('How Game Length Changes Strategy Rankings')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

In this tutorial, we learned:

1. **Nash equilibrium** — the only stable strategy in one-shot PD is mutual defection
2. **Dominant strategies** — defection dominates regardless of what the opponent does
3. **Repeated games change everything** — tit-for-tat and cooperative strategies outperform defection when games have history
4. **The Folk Theorem** — cooperation is self-enforcing when δ ≥ (T-R)/(T-P)
5. **Noise favors forgiveness** — in imperfect environments (near the Dens), generous strategies outperform strict ones

**Key insight:** Game theory tells us *when* cooperation can be sustained (repeated games, long time horizons, noise tolerance). Mechanism design — the subject of Tutorial 03 — tells us *how to engineer the rules* so that cooperation is inevitable.

---

**Next Tutorial:** Mechanism Design — reverse game theory: design the game to get the outcome you want.

---

> *"Game theory describes the prison. Mechanism design builds the door."*
>
> — Brenn Auster